In [1]:
from ROOT import TFile,TTree,TH1F,gStyle,TLatex,TCanvas,TGraphErrors,TLegend,TH2F,TLine,TGraph,gPad,TGraph2D, TF1, TFitResultPtr
import numpy as np
import pandas as pd
from scipy import integrate
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import make_interp_spline

Welcome to JupyROOT 6.24/02


In [31]:
# -------------------------------
# 
# DEUTERON MODEL 
#
# -------------------------------

# Load the Model
df=pd.read_csv("harrymodel/JPsi_d.csv",sep="\t")
df=df.sort_values(by=['Beam Energy [GeV]','-t [GeV**2]'])
dfV=df.values
df_E=dfV[:,1]
df_T=dfV[:,2]
df_DSDT=dfV[:,3]
interp_d=LinearNDInterpolator(list(zip(df_E,df_T)),df_DSDT)

# Get tmin and tmax for each energy
tmtm = np.zeros((np.unique(df_E).size,5))
idxtmp=0
for tmpE in np.unique(df_E):
    tminTemp=df_T[np.where(df_E==tmpE)[0][0]]
    tmaxTemp=df_T[np.where(df_E==tmpE)[0][-1]]
    idx1=np.where(df_E==tmpE)[0][0]
    idx2=np.where(df_E==tmpE)[0][-1]
    tmtm[idxtmp]=(tmpE,tminTemp,tmaxTemp,idx1,idx2)
    idxtmp+=1

def get_tmtm(gammaE):
    idx_of_E = np.abs(gammaE - tmtm[:,0]).argmin()
    return tmtm[idx_of_E,0],tmtm[idx_of_E,1],tmtm[idx_of_E,2]

# Get dsdt from interpolation
def dsdt_d(gammaE,t):
    return interp_d(gammaE,t)

# Get dsdt from interpolation
def dsdt2_d(Egmin,Egmax,tmin,tmax):
    mySum = 0.0
    niter = 1000
    
    count = 0
    for i in range(niter):
        eg = np.random.uniform(Egmin,Egmax)
        t = np.random.uniform(tmin,tmax)
        dsdt = dsdt_d(eg,t)
        if(not np.isnan(dsdt)):
            mySum+=dsdt
            count+=1
    if(count==0):
        return 0
    else:
        return mySum/count
    
def xsec(gammaE):
    if(gammaE<=6):
        idx=int(tmtm[tmtm[:,0] >= gammaE].min())
        DSDT_arr=df_DSDT[int(tmtm[idx,3]):int(tmtm[idx,4])]
        T_arr=df_T[int(tmtm[idx,3]):int(tmtm[idx,4])]
        return integrate.simpson(DSDT_arr,T_arr)
    gammaE_0=tmtm[tmtm[:,0] < gammaE,0].max()
    gammaE_1=tmtm[tmtm[:,0] > gammaE,0].min()
    diff=(gammaE-gammaE_0)/(gammaE_1-gammaE_0)
    idx_0=list(tmtm[:,0]).index(gammaE_0)
    idx_1=list(tmtm[:,0]).index(gammaE_1)
    DSDT_arr_0 = df_DSDT[int(tmtm[idx_0,3]):int(tmtm[idx_0,4])]
    T_arr_0 = df_T[int(tmtm[idx_0,3]):int(tmtm[idx_0,4])]
    DSDT_arr_1 = df_DSDT[int(tmtm[idx_1,3]):int(tmtm[idx_1,4])]
    T_arr_1 = df_T[int(tmtm[idx_1,3]):int(tmtm[idx_1,4])]
    diff=1-(gammaE-gammaE_0)/(gammaE_1-gammaE_0)
    return diff*integrate.simpson(DSDT_arr_0,T_arr_0)+(1-diff)*integrate.simpson(DSDT_arr_1,T_arr_1)

In [4]:
def get_tmin(Eg):
    W = np.sqrt(2*1.875612928*Eg+1.875612928**2)
    m1_2 = 0;
    m2_2 = np.power(1.875612928,2);
    m3_2 = np.power(3.0969,2);
    m4_2 = np.power(1.875612928,2);
    p1 = np.sqrt(np.power(W * W - m1_2 - m2_2,2) - 4.0 * m1_2 * m2_2) / (2.0 * W);
    p2 = np.sqrt(np.power(W * W - m3_2 - m4_2,2) - 4.0 * m3_2 * m4_2) / (2.0 * W);
    return m2_2 + m4_2 - 2 * np.sqrt(m2_2 + p1*p1) * np.sqrt(m4_2 + p2*p2) + 2 * p1 * p2;

def get_tmax(Eg):
    W = np.sqrt(2*1.875612928*Eg+1.875612928**2)
    m1_2 = 0;
    m2_2 = np.power(1.875612928,2);
    m3_2 = np.power(3.0969,2);
    m4_2 = np.power(1.875612928,2);
    p1 = np.sqrt(np.power(W * W - m1_2 - m2_2,2) - 4.0 * m1_2 * m2_2) / (2.0 * W);
    p2 = np.sqrt(np.power(W * W - m3_2 - m4_2,2) - 4.0 * m3_2 * m4_2) / (2.0 * W);
    return m2_2 + m4_2 - 2 * np.sqrt(m2_2 + p1*p1) * np.sqrt(m4_2 + p2*p2) - 2 * p1 * p2;


In [82]:
def get_dsigdt(tree, lumi, nEvents, q2cut, tVals, Egmin, Egmax):
    
    nPoints = len(tVals)-1
    
    Egmid = (Egmax+Egmin)/2.0
    tmin_tmp  = -get_tmin(Egmid)
    # Setup TGraphErrors
    tg = TGraphErrors(nPoints)
    for i in range(nPoints):
        tx = np.mean(tVals[i:i+2])
        #tg.SetPoint(i+1, tx , dsdt_d(Egmid,tx+tmin_tmp))
        tg.SetPoint(i+1, tx , dsdt2_d(Egmin,Egmax,tVals[i]+tmin_tmp,tVals[i+1]+tmin_tmp))
        
        
        print(tx,tVals[i]+tmin_tmp,tVals[i+1]+tmin_tmp,Egmin,Egmax,tg.GetPointY(i+1))
        #print(tx,tmin_tmp,Egmid,tx+tmin_tmp,dsdt_d(Egmid,tx+tmin_tmp))
        #if(i==nPoints-1):
        #    tg.SetPointError(i+1, 0.5*(tVals[i]-tVals[i-1]) , 0 )
        #else:
        #    tg.SetPointError(i+1, 0.5*(tVals[i+1]-tVals[i]) , 0 )
        tg.SetPointError(i+1, 0.5*(tVals[i+1]-tVals[i]) , 0 )
        
    # Loop over Tree
    for iev in tree:
        
        # Toss events where acceptance is not made
        acc_1 = iev.acc_hOut
        acc_2 = iev.acc_ePlus
        acc_3 = iev.acc_eMinus
        if(acc_1 * acc_2 * acc_3 == 0):
            continue
        # Toss events where Eg is not in range
        Eg = iev.gammaE
        if(Eg<Egmin or Eg>Egmax):
            continue
        
        # Toss events where Q2 is not in range
        smear_q = iev.smear_q
        Q2 = smear_q*smear_q * (-1)
        if(Q2>q2cut or Q2<-q2cut):
            continue
        
        # Get t of event, and tlimits
        tmin = get_tmin(Eg)
        tmax = get_tmax(Eg)
        t = -(iev.t - tmin)
        
        # Find the TGraph index of the t
        tg_idx=(int)(np.searchsorted(tVals,t))
        
        if(tg_idx>nPoints or tg_idx==0):
            continue  
        # Calculate weight for event
        weight = iev.weight
        psf = iev.psf
        flux = iev.flux
        decay_weight = iev.decay_weight
        total_weight = weight*psf*flux*decay_weight*lumi/nEvents
        
        # Increment the yerr to reflect nEvents within that bin
        tg.SetPointError(tg_idx, tg.GetErrorX(tg_idx), tg.GetErrorY(tg_idx) + total_weight)
    
    # Reconfigure TGraphErrors for realistic error bars
    
    for i in range(nPoints):
        dsdt = tg.GetPointY(i+1)
        N = tg.GetErrorY(i+1)
        print(N)
        if(N==0):
            tg.SetPointError(i+1, tg.GetErrorX(i+1), 0)
        else:
            tg.SetPointError(i+1, tg.GetErrorX(i+1) , dsdt/np.sqrt(N) )
            
    return tg

In [5]:
def get_dsigdts(tree, lumi, nEvents, q2cut, tVals, Egs):
    
    nGraphs = len(Egs)-1
    
    tgs = []
    
    for i in range(nGraphs):
        print("Spawning TGraph {}/{}".format(i+1,nGraphs))
        Egmid = (Egs[i]+Egs[i+1])/2.0
        tmin_tmp  = -get_tmin(Egmid)
        nPoints = len(tVals[i])-1
        # Setup TGraphErrors
        tg = TGraphErrors(nPoints)
        for j in range(nPoints):
            tx = np.mean(tVals[i][j:j+2])
            tg.SetPoint(j, tx , dsdt2_d(Egs[i],Egs[i+1],tVals[i][j]+tmin_tmp,tVals[i][j+1]+tmin_tmp))
            tg.SetPointError(j, 0.5*(tVals[i][j+1]-tVals[i][j]) , 0 )
        tgs.append(tg)
    
    counter = 0
    nentries= tree.GetEntriesFast()
    
    # Loop over Tree
    for iev in tree:
        if(counter%100000==0):
            print("Completed {} events out of {} in TTree".format(counter,nentries))
        counter+=1
        # Toss events where acceptance is not made
        acc_1 = iev.acc_hOut
        acc_2 = iev.acc_ePlus
        acc_3 = iev.acc_eMinus
        if(acc_1 * acc_2 * acc_3 == 0):
            continue
            
        # Toss events where Eg is not in range
        Eg = iev.gammaE
        if(Eg<Egs[0] or Eg>Egs[nGraphs]):
            continue
        
        # Find the TGraph index of the Eg
        Eg_idx = (int)(np.searchsorted(Egs,Eg))-1
        
        # Toss events where Q2 is not in range
        smear_q = iev.smear_q
        Q2 = smear_q*smear_q * (-1)
        if(Q2>q2cut or Q2<-q2cut):
            continue
        
        # Toss events where smear_VM mass is not in range
        smear_VM = iev.smear_VM
        if(smear_VM.M()<3.0 or smear_VM.M() > 3.2):
            continue
            
        
        # Get t of event, and tlimits
        tmin = get_tmin(Eg)
        tmax = get_tmax(Eg)
        t = -(iev.t - tmin)
        
        # Find the TGraph index of the t
        t_idx=(int)(np.searchsorted(tVals[Eg_idx],t))-1
        if(t_idx>nPoints or t_idx<0):
            continue  
        # Calculate weight for event
        weight = iev.weight
        psf = iev.psf
        flux = iev.flux
        decay_weight = iev.decay_weight
        total_weight = weight*psf*flux*decay_weight*lumi/nEvents
        
        # Increment the yerr to reflect nEvents within that bin
        tgs[Eg_idx].SetPointError(t_idx, tgs[Eg_idx].GetErrorX(t_idx), tgs[Eg_idx].GetErrorY(t_idx) + total_weight)
        
    tgs_final=[]
    # Reconfigure TGraphErrors for realistic error bars
    for tg in tgs:
        nPoints = tg.GetN()
        tg_final=TGraphErrors()
        k = 0
        for j in range(nPoints):
            dsdt = tg.GetPointY(j)
            N = tg.GetErrorY(j)
            if(N!=0):
                #print("A")
                tg_final.AddPoint(tg.GetPointX(j),tg.GetPointY(j))
                tg_final.SetPointError(k, tg.GetErrorX(j) , dsdt*np.sqrt(1/N+0.2**2) )
                k+=1
        tgs_final.append(tg_final)
    
    print("---------------------")
    print("Done \n")
    return tgs_final

In [6]:
def do_fits(tgs,Egs):
    A = []
    oA = []
    B = []
    oB = []
    chi2 = []
    ndf = []
    integral = []
    integral_error = []
    
    exp_fit = TF1("exp_fit","expo",0,3,2)
    
    i = 0
    for tg in tgs:
        exp_fit.SetParameters(0.008,-10)
        fitresultptr = tg.Fit(exp_fit,"NQ0S")
        A.append(np.exp(exp_fit.GetParameter(0)))
        B.append(exp_fit.GetParameter(1))
        oA.append(exp_fit.GetParError(0)*np.exp(exp_fit.GetParameter(0)))
        oB.append(exp_fit.GetParError(1))
        chi2.append(exp_fit.GetChisquare())
        ndf.append(exp_fit.GetNDF())
        
        Egmid = (Egs[i]+Egs[i+1])/2.0
        tmin = -get_tmin(Egmid)
        tmax = -get_tmax(Egmid)
        
        if(tg.GetN()<=1):
            integral.append(-999)
            integral_error.append(-999)
        else:
            integral.append(exp_fit.Integral(0,tmax-tmin))
            integral_error.append(exp_fit.IntegralError(0,tmax-tmin,fitresultptr.GetParams(), fitresultptr.GetCovarianceMatrix().GetMatrixArray()))
        i+=1
    return A,oA,B,oB,chi2,ndf, integral, integral_error

In [38]:
def drawLatex(i,beamE,days,EgArray,A,oA,B,oB,chi2,ndf,integral,integral_error,latex,biglatex):
    if(i==2):
        latex.DrawLatexNDC(.5,.95,"#it{SoLID} MC Simulation")
        latex.DrawLatexNDC(.67,.9,"#color[4]{3-fold coincidence}")
        latex.DrawLatexNDC(.5,.85,"e({:.1f}GeV)+d #rightarrow #gamma+d #rightarrow ".format(beamE) + "#color[4]{d'} + J/#psi#color[4]{(e-e+)}")
        latex.DrawLatexNDC(.5,.8,"{} days @ L = 1.2e37 nucleons/".format(days)+"cm^{2}/s")
        latex.DrawLatexNDC(.55,.75,"-0.02 < Q^{2}_{reco} < 0.02 GeV^{2}")
        latex.DrawLatexNDC(.55,.7,"3.0 < M_{ll}^{2} < 3.2 GeV^{2}")
        latex.DrawLatexNDC(.55,.65,"#delta_{d#sigma/dt} = d#sigma/dt #times #sqrt{#frac{1}{N} + (20%)^{2}}")
    
    
    if(i!=2):
        latex.DrawLatexNDC(0.67,0.95,"f(t)= #color[2]{A} e^{#color[2]{B}t}")
        latex.DrawLatexNDC(0.67,0.9,"#color[2]{A} = "+"({:.1f}#pm{:.1f})".format(A[i]*1e3,oA[i]*1e3)+"#times10^{-3}")
        latex.DrawLatexNDC(0.67,0.85,"#color[2]{B} = "+"{:.2f}#pm{:.2f}".format(B[i],oB[i]))
        latex.DrawLatexNDC(0.67,0.8,"#chi^{2}/ndf = "+"{:.2f}/{:.0f} = {:.2f}".format(chi2[i],ndf[i],chi2[i]/ndf[i]))
    else:
        latex.DrawLatexNDC(0.05,0.95,"f(t)= #color[2]{A} e^{#color[2]{B}t}")
        latex.DrawLatexNDC(0.05,0.9,"#color[2]{A} = "+"({:.1f}#pm{:.1f})".format(A[i]*1e3,oA[i]*1e3)+"#times10^{-3}")
        latex.DrawLatexNDC(0.05,0.85,"#color[2]{B} = "+"{:.2f}#pm{:.2f}".format(B[i],oB[i]))
        latex.DrawLatexNDC(0.05,0.8,"#chi^{2}/ndf = "+"{:.2f}/{:.0f} = {:.2f}".format(chi2[i],ndf[i],chi2[i]/ndf[i]))
    
    biglatex_x = 0
    biglatex_y = 0
    
    if(i==0 or i==1 or i==2):
        biglatex_y = 0.075
    else:
        biglatex_y = 0.15
        
    if(i==0 or i==3):
        biglatex_x = 0.2
    else:
        biglatex_x = 0.1
    biglatex.DrawLatexNDC(biglatex_x,biglatex_y,"{:.2f} < ".format(EgArray[i]) + "E_{#gamma} < " + "{:.2f} GeV".format(EgArray[i+1]))
    return

In [8]:
def get_sigma(Egmid,integral,integral_error):
    tge = TGraphErrors(len(Egmid))
    for i in range(len(Egmid)):
        tge.SetPoint(i,Egmid[i],integral[i])
        tge.SetPointError(i,0,integral_error[i])
    return tge

In [15]:
'''def get_true_sigma(Egmid):
    xsec_graph=TGraph(len(Egmid)-1)
    for i in range(xsec_graph.GetN()):
        Egbin = np.array([Egmid[i],Egmid[i+1]])
        xsec_graph.SetPoint(i,np.mean(Egbin), x)
    return xsec_graph'''

In [25]:
def get_true_sigma():
    g_arr = np.array([6,7,8,9,10,12])
    xsec_arr = [xsec(g) for g in g_arr]
    X_Y_Spline = make_interp_spline(g_arr, xsec_arr)

    npoints2=100
    g_arr2 = np.linspace(6,12,npoints2)
    xsec_arr2 = X_Y_Spline(g_arr2)
    xsec_graph=TGraph(npoints2)
    for i in range(npoints2):
        xsec_graph.SetPoint(i,g_arr2[i],xsec_arr2[i])
    xsec_graph.SetLineColor(2)
    return xsec_graph

In [28]:
xsec(7)

2.616409280993664e-05